# Day 1 Problems

### Avishek Shaw

In [14]:
from pulp import *

# Exercise 1: Coin Production Problem

#### Let us first analyse this problem.    
a) No. of optimization variables: 5   
b) No. of inequality constraints: 4   
c) No. of slack variables: 4   
d) Dimensionality of the basis: 4   

Hence, we expect that **only 4 out of the 9 variables (slack+opt) will have non zero values**     
   
Equivalently, one could say that the solution vector (1000,50,50,50) can be uniquely determined by a linear combination of 4 nos. of 4-d vectors only, provided, the solution vector can be formed in the vector space of the chosen 4 vectors

#### data model

In [15]:
coin_values = { "penny":0.01, "nickel":0.05, "dime":0.10, "quarter":0.25, "dollar":1.0}

In [16]:
metal_limits = {"copper":1000, "nickel":50, "zinc":50, "manganese":50 }

In [17]:
composition = {
    "copper":    {"penny":0.06, "nickel":3.8, "dime":2.1, "quarter":5.2, "dollar":7.2},
    "nickel":    {"penny":0.00, "nickel":1.2, "dime":0.2, "quarter":0.5, "dollar":0.2},
    "zinc":      {"penny":2.40, "nickel":0.0, "dime":0.0, "quarter":0.0, "dollar":0.5},
    "manganese": {"penny":0.00, "nickel":0.0, "dime":0.0, "quarter":0.0, "dollar":0.3}
}

#### optimization model

In [18]:
# Initialize the model
opt_model = LpProblem(name="Coin Production problem", sense=LpMaximize)

/home/avishek/anaconda3/envs/opt/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [19]:
# Decision Variables
x = LpVariable.dicts("", coin_values, lowBound=0, cat=LpContinuous)

In [20]:
# Objective function
opt_model += lpSum(coin_values[coin] * x[coin] for coin in coin_values)

In [21]:
# Constraints
for metal in metal_limits:
    opt_model += lpSum(composition[metal][coin] * x[coin] for coin in coin_values)\
    <= metal_limits[metal]

In [22]:
# check the model
opt_model

Coin_Production_problem:
MAXIMIZE
0.1*_dime + 1.0*_dollar + 0.05*_nickel + 0.01*_penny + 0.25*_quarter + 0.0
SUBJECT TO
_C1: 2.1 _dime + 7.2 _dollar + 3.8 _nickel + 0.06 _penny + 5.2 _quarter
 <= 1000

_C2: 0.2 _dime + 0.2 _dollar + 1.2 _nickel + 0.5 _quarter <= 50

_C3: 0.5 _dollar + 2.4 _penny <= 50

_C4: 0.3 _dollar <= 50

VARIABLES
_dime Continuous
_dollar Continuous
_nickel Continuous
_penny Continuous
_quarter Continuous

In [23]:
# writing to LP file
opt_model.writeLP("Coin Production problem")

[_dime, _dollar, _nickel, _penny, _quarter]

In [24]:
# Solver
solver = getSolver('COIN_CMD')
opt_model.solve(solver)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Dec  8 2020 

command line - cbc /tmp/38b29c9b6e1f4dc1ac2aec061e02398a-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/38b29c9b6e1f4dc1ac2aec061e02398a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 27 RHS
At line 32 BOUNDS
At line 33 ENDATA
Problem MODEL has 4 rows, 5 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-1) rows, 5 (0) columns and 11 (-1) elements
0  Obj -0 Dual inf 0.83515034 (5)
3  Obj 113.46154
Optimal - objective value 113.46154
After Postsolve, objective 113.46154, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 113.4615385 - 3 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



1

In [25]:
print("Model Status: " + str(LpStatus[opt_model.status]))
print("Model Objective " + str(value(opt_model.objective)))

Model Status: Optimal
Model Objective 113.4615385


In [26]:
solution = {}
for coin in coin_values:
    solution[coin] = x[coin].varValue
solution

{'penny': 0.0,
 'nickel': 0.0,
 'dime': 0.0,
 'quarter': 53.846154,
 'dollar': 100.0}

In [27]:
# Slack and Dual Values
print("")
print("Slack and Dual Variables")
print("")
for name,c in list(opt_model.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)
print("")


Slack and Dual Variables

_C1 : 2.1*_dime + 7.2*_dollar + 3.8*_nickel + 0.06*_penny + 5.2*_quarter <= 1000.0 	 0.048076923 		 -0.0
_C2 : 0.2*_dime + 0.2*_dollar + 1.2*_nickel + 0.5*_quarter <= 50.0 	 -0.0 		 3.0769230000000007
_C3 : 0.5*_dollar + 2.4*_penny <= 50.0 	 1.3076923 		 -0.0
_C4 : 0.3*_dollar <= 50.0 	 -0.0 		 20.0



Our estimate earlier was correct, 2 optimization variables (quarter, dollar) and 2 slack variables (C2,C4) are non zero. Rest all of them are zero.
Also, the solution is interesting, the solver is maximizing for dollars as it has a disproportionate advantage over other coins

# Exercise 2: Advertising Budget Problem

**Let us first analyse this problem**   
a) No. of optimization variables: 2  
b) No. of inequality constraints: 2   
c) No. of slack variables: 2   
d) Dimensionality of the basis: 2   

Hence, we expect that **only 2 out of the 4 variables (slack+opt) will have non zero values**     

#### data model

In [28]:
theme = {"theme_1":1, "theme_2":1}
customer_targets = {"business":50, "leisure":100}
acquisition = {
    "theme_1": {"business":2, "leisure": 10},
    "theme_2": {"business":8, "leisure": 1}
}

In [29]:
# model initialization
opt_model = LpProblem("Advertising_Budget", LpMinimize)

In [30]:
# Decision variables
x = LpVariable.dicts("",theme,lowBound=0,cat=LpContinuous)

In [31]:
# Objective function
opt_model += lpSum(x[i] for i in theme)

In [32]:
# Constraints
for target in customer_targets:
#     print(acquisition[th][target] for th in theme)
    opt_model += lpSum(acquisition[th][target] * x[th] for th in theme) >= customer_targets[target]

In [33]:
# check the model
opt_model

Advertising_Budget:
MINIMIZE
1*_theme_1 + 1*_theme_2 + 0
SUBJECT TO
_C1: 2 _theme_1 + 8 _theme_2 >= 50

_C2: 10 _theme_1 + _theme_2 >= 100

VARIABLES
_theme_1 Continuous
_theme_2 Continuous

In [34]:
# writing to LP file
opt_model.writeLP("Advertising Budget Problem")

[_theme_1, _theme_2]

In [35]:
# Solver
solver = getSolver('COIN_CMD')
opt_model.solve(solver)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Dec  8 2020 

command line - cbc /tmp/186bc2727c21477ba24bda8f737e0e4f-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/186bc2727c21477ba24bda8f737e0e4f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 14 RHS
At line 17 BOUNDS
At line 18 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 2 (0) columns and 4 (0) elements
0  Obj 0 Primal inf 16.25 (2)
2  Obj 13.461538
Optimal - objective value 13.461538
Optimal objective 13.46153846 - 2 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



1

In [36]:
print("Model Status: " + str(LpStatus[opt_model.status]))
print("Model Objective " + str(value(opt_model.objective)))

Model Status: Optimal
Model Objective 13.4615384


In [37]:
solution = {}
for th in theme:
    solution[th] = x[th].varValue
solution

{'theme_1': 9.6153846, 'theme_2': 3.8461538}

In [38]:
# Slack and Dual Values
print("")
print("Slack and Dual Variables")
print("")
for name,c in list(opt_model.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)
print("")


Slack and Dual Variables

_C1 : 2*_theme_1 + 8*_theme_2 >= 50 	 0.11538462 		 -0.0
_C2 : 10*_theme_1 + _theme_2 >= 100 	 0.076923077 		 -0.0



As the leisure requirement is more, the solution is dominated by its requirement. i.e. we have more spending on theme1 than theme2